In [ ]:
# Write fake accuracies
import tensorflow as tf

writer = tf.summary.create_file_writer('./graphs')

accuracy = [0.1, 0.4, 0.6, 0.8, 0.9, 0.95] # fake accuracy values

with writer.as_default():
    for step, acc in enumerate(accuracy):
        tf.summary.scalar('Accuracy', acc, step) # add summary
        writer.flush() # make sure everything is written to disk

writer.close() # not really needed, but good habit

In [ ]:
# Visualize a basic graph

"""
As TensorFlow 2.0 generally operates in eager mode, there is often no graph created, and thus no way to
visualize it.

The following is a possible workaround using @tf.function. 
"""

import tensorflow as tf
from tensorflow.python.ops import summary_ops_v2

# Graph
a = tf.Variable(2, name='a')
b = tf.Variable(3, name='b')

@tf.function
def graph_to_visualize(a, b):
    c = tf.add(a, b, name='Add')

# Visualize
writer = tf.summary.create_file_writer('./graphs')

with writer.as_default():
    graph = graph_to_visualize.get_concrete_function(a, b).graph # get graph from function
    summary_ops_v2.graph(graph.as_graph_def()) # visualize

writer.close()

In [ ]:
# Visualize a giant graph
import tensorflow as tf

a = tf.Variable(2.0, name='a')
b = tf.Variable(3.0, name='b')
c = tf.Variable(7.0, name='c')

@tf.function
def graph_to_visualize(a, b, c):
    d = tf.multiply(a, b, name='d-mul')
    e = tf.add(b, c, name='e-add')
    f = tf.subtract(e, a, name='f-sub')
    g = tf.multiply(d, b, name='g-mul')
    h = tf.divide(g, d, name='h-div')
    i = tf.add(h, f, name='i-add')

writer = tf.summary.create_file_writer('./graphs')

with writer.as_default():
    graph = graph_to_visualize.get_concrete_function(a, b, c).graph # get graph from function
    summary_ops_v2.graph(graph.as_graph_def()) # visualize

writer.close()

In [ ]:
# Visualize a U-Net generator
"""Using the previous techniques, it is needlessly hard to visualize a Keras model.
Instead, use the Keras TensorBoard callback when compiling your model.

We will talk more about this later.
"""

In [ ]:
# Make a simple normal distribution
import tensorflow as tf

normal_dist = tf.random.normal(shape=(10, 10, 10, 10, 10), mean=0, stddev=1) # tensor to be logged, shape is irrelevant

writer = tf.summary.create_file_writer('./graphs')
with writer.as_default():
    tf.summary.histogram("NormalDistribution", normal_dist, step=0) # log tensor
    writer.flush()

writer.close()

In [ ]:
# Stack 100 normal distributions
import tensorflow as tf

writer = tf.summary.create_file_writer('./graphs')

with writer.as_default():
    for i in range(100):
        normal_dist = tf.random.normal(shape=(10, 10, 10, 10, 10), mean=i/50, stddev=1) # tensor to be logged, shape is irrelevant
        tf.summary.histogram("NormalDistribution", normal_dist, step=i) # summary that logs tensor

        writer.flush()
writer.close()

In [ ]:
# Visualize a multimodal distributon
import tensorflow as tf

writer = tf.summary.create_file_writer('./graphs')

with writer.as_default():
    for i in range(100):
        normal_dist_0 = tf.random.normal(shape=(10000,), mean=-i/20, stddev=0.5)
        normal_dist_1 = tf.random.normal(shape=(10000,), mean=i/20, stddev=0.5)
        normal_dist_cat = tf.concat([normal_dist_0, normal_dist_1], axis=0) # concatenate both normal dists

        tf.summary.histogram("MultimodalDistribution", normal_dist_cat, step=i) # summary that logs tensor

writer.flush()
writer.close()

In [ ]:
# Visualize some random nosie
import tensorflow as tf

image = tf.random.uniform((1, 210, 160, 3), 0, 1, dtype=tf.float32) # batch_size, height, width, channels
writer = tf.summary.create_file_writer('./graphs')

with writer.as_default():
    img_summary = tf.summary.image("InputImage", image, step=0)
    writer.flush()

writer.close()

In [ ]:
# Visuaize the first 4 examples of MNIST
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

images = x_train[:4].reshape((-1, 28, 28, 1))
writer = tf.summary.create_file_writer('./graphs')

with writer.as_default():
    tf.summary.image("TrainingExamples", images, max_outputs=4, step=0)

    writer.flush()
writer.close()

In [ ]:
# Display some text
import tensorflow as tf

text = tf.convert_to_tensor('Hello world!')
writer = tf.summary.create_file_writer('./graphs')

with writer.as_default():
    tf.summary.text("Text", text, step=0)

    writer.flush()
writer.close()

In [ ]:
# Merge summaries
"""There is no equivalent to tf.summary.merge in TensorFlow 2.0"""

In [ ]:
# Merge all summaries
"""There is no equivalent to tf.summary.merge_all in TensorFlow 2.0"""

In [ ]:
# Use the TensorBoard callback in a Keras model
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128), # we will visualize neurons in this layer
    Dropout(0.5),
    Dense(10, activation='softmax')
])

tensorboard_callback = tf.keras.callbacks.TensorBoard('./graphs',
    histogram_freq=1, # how often to compute activation and weight histograms
    write_graph=True, # visualize the graph
    write_grads=True, # visual gradient histogram
    write_images=True, # visualize model weights as an image
    # embeddings_freq=1, # how often to visualize embeddings
    # embeddings_layer_names=['...'], # names of embedding layers to visualize; wouldn't work on this model
    update_freq='epoch' # update TensorBoard every epoch
)

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=32,
          epochs=15,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])